In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

polars.config.Config

In [3]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [5]:
entry_date = dt.date(2023, 7, 27)
expiry = get_expiry(entry_date)
expiry

datetime.date(2023, 7, 27)

In [6]:
option = await fetch_option_data(
    index='bnf',
    expiry=expiry,
    strike=45900,
    asset_class=AssetClass.CALL,
    start_date=entry_date,
    start_time=dt.time(9, 15),
    end_date=entry_date,
    end_time=dt.time(15, 30)
)
# option

In [411]:
# bnf = await fetch_spot_data(
#     instrument='bnf',
#     start_date=dt.date(2022, 1, 3),
#     start_time=dt.time(9, 15),
#     end_date=dt.date(2023, 1, 5),
#     end_time=dt.time(15, 30),
# )
# bnf = bnf.to_pandas()
# bnf

# bnf = pd.read_csv('../data/bnf_daily.csv')
bnf = pd.read_csv('../data/bnf_min.csv')
bnf = pl.DataFrame(bnf)
bnf = bnf.with_columns(pl.col('datetime').str.to_datetime(format='%Y-%m-%dT%H:%M:%S.%f'))
bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# bnf

/tmp/ipykernel_200979/3418792131.py:14: ChronoFormatWarning:

Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime



open,high,low,close,volume,index,datetime
f64,f64,f64,f64,i64,datetime[ns],datetime[ns]
18242.3,18248.2,18175.9,18181.2,0,2017-01-02 09:15:00,2017-01-02 09:15:00
18181.85,18194.7,18179.95,18184.45,0,2017-01-02 09:16:00,2017-01-02 09:16:00
18184.95,18189.25,18133.8,18133.8,0,2017-01-02 09:17:00,2017-01-02 09:17:00
18135.1,18141.55,18118.55,18138.95,0,2017-01-02 09:18:00,2017-01-02 09:18:00
18138.95,18142.55,18120.45,18124.3,0,2017-01-02 09:19:00,2017-01-02 09:19:00
18123.95,18123.95,18100.3,18100.3,0,2017-01-02 09:20:00,2017-01-02 09:20:00
18101.65,18112.15,18091.45,18112.1,0,2017-01-02 09:21:00,2017-01-02 09:21:00
18114.35,18116.2,18091.65,18101.6,0,2017-01-02 09:22:00,2017-01-02 09:22:00
18103.65,18115.1,18094.8,18102.9,0,2017-01-02 09:23:00,2017-01-02 09:23:00


In [412]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            truncate=True,
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

ohlc_resampled = resample(bnf, '15m', pd.Timedelta(minutes=0))

bnf_1hr = ohlc_resampled
bnf_final = bnf_1hr.to_pandas()
bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final

/tmp/ipykernel_200979/3688010034.py:13: DeprecationWarning:

`truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.



In [413]:
print(type(bnf))

<class 'polars.dataframe.frame.DataFrame'>


In [414]:
bnf = bnf_final.rename(columns={'open': 'o', 'high': 'h', 'low': 'l', 'close': 'c'})
# bnf = bnf.to_pandas()
# bnf = bnf.rename(columns={'open': 'o', 'high': 'h', 'low': 'l', 'close': 'c'})
# bnf

,datetime,o,h,l,c,volume
0,2017-01-02 09:15:00,18242.30,18248.20,18052.15,18052.15,0
1,2017-01-02 09:30:00,18051.25,18052.35,17995.40,18049.80,0
2,2017-01-02 09:45:00,18048.25,18048.55,18003.60,18020.60,0
3,2017-01-02 10:00:00,18019.65,18040.95,18007.90,18017.60,0
4,2017-01-02 10:15:00,18017.10,18035.75,18014.45,18014.70,0
...,...,...,...,...,...,...
42888,2024-01-31 14:15:00,46041.70,46054.20,45922.35,45958.50,0
42889,2024-01-31 14:30:00,45962.00,46075.45,45948.10,46062.10,0
42890,2024-01-31 14:45:00,46068.00,46137.80,46001.55,46032.15,0
42891,2024-01-31 15:00:00,46038.95,46053.85,45942.45,45990.60,0


In [493]:
import numpy as np

def calculate_psar(high, low, close, start=0.02, inc=0.02, max_value=0.2):
    psar = np.zeros_like(close, dtype=float)
    psar[0] = low[0]  # Initial PSAR value is the first low

    # Initial values
    ep = high[0]
    af_current = start
    is_below = True  # Assuming downward trend initially
    
    for i in range(1, len(close)):
        psar[i] = psar[i - 1] + af_current * (ep - psar[i - 1])
        
        if is_below:
            if high[i] > ep:
                ep = high[i]
                af_current = min(af_current + inc, max_value)
            if low[i] < psar[i]:
                is_below = False
                psar[i] = ep
                ep = low[i]
                af_current = start
        else:
            if low[i] < ep:
                ep = low[i]
                af_current = min(af_current + inc, max_value)
            if high[i] > psar[i]:
                is_below = True
                psar[i] = ep
                ep = high[i]
                af_current = start

    df['MA'] = df['c'].rolling(window=13).mean()
    # df.dropna(inplace=True)
    return psar

# Example usage:
# Assuming 'df' is your DataFrame containing OHLC data
# psar_values = calculate_psar(df['High'].values, df['Low'].values, df['Close'].values)
# df['PSAR'] = psar_values
# print(df)


In [494]:
df = pd.DataFrame(bnf)

psar_values = calculate_psar(df['h'].values, df['l'].values, df['c'].values)

df['psar'] = psar_values

df.dropna(inplace=True)

# df

In [495]:
def strategy(df):
    tradebook = []
    in_trade = False
    entry_price = None
    trade_type = None
    exit_price = None
    is_trailing_active = False
    initial_sl = None

    for i in range(1, len(df)):
        # Check for buy signal
        if df.iloc[i]['psar'] < df.iloc[i]['c'] and df.iloc[i-1]['psar'] > df.iloc[i-1]['c']:
            buy_signal = 1
        else:
            buy_signal = 0

        # Check for sell signal
        if df.iloc[i]['psar'] > df.iloc[i]['c'] and df.iloc[i-1]['psar'] < df.iloc[i-1]['c']:
            sell_signal = 1
        else:
            sell_signal = 0

        if not in_trade:
            # Entry conditions
            if buy_signal == 1:
                in_trade = True
                entry_time = df.iloc[i]['datetime']
                entry_price = df.iloc[i]['c']
                trade_type = 'Buy'
                initial_sl = min(df.iloc[i]['l'], df.iloc[i-1]['l'], df.iloc[i-2]['l'])
            elif sell_signal == 1:
                in_trade = True
                entry_time = df.iloc[i]['datetime']
                entry_price = df.iloc[i]['c']
                trade_type = 'Sell'
                initial_sl = max(df.iloc[i]['h'], df.iloc[i-1]['h'], df.iloc[i-2]['h'])

        else:
            if in_trade:
                
                if not is_trailing_active and trade_type == 'Buy' and df.iloc[i]['l'] > df.iloc[i]['MA']:
                    #Activate Trailing
                    is_trailing_active = True
                elif not is_trailing_active and trade_type == 'Sell' and df.iloc[i]['h'] < df.iloc[i]['MA']:
                    #Activate Trailing
                    is_trailing_active = True

                # if in_trade and trade_type == "Buy" and df.iloc[i]['datetime'].time() >= dt.time(15, 15):
                #     # EOD Square Off
                #     exit_price = df.iloc[i]['c']
                #     exit_time = df.iloc[i]['datetime']
                #     remark = 'EOD Sq Off'
                #     trade = {
                #         'Entry Time': entry_time,
                #         'Entry Price': entry_price,
                #         'Initial SL': initial_sl,
                #         'Exit Time': exit_time,
                #         'Exit Price': exit_price, 
                #         'Profit': exit_price - entry_price,
                #         'Remark': remark,
                #         'Trade Type': trade_type,
                #     }
                #     tradebook.append(trade)
                #     in_trade = False
                #     is_trailing_active = False

                # elif in_trade and trade_type == "Sell" and df.iloc[i]['datetime'].time() >= dt.time(15, 15):
                #     # EOD Square Off
                #     exit_price = df.iloc[i]['c']
                #     exit_time = df.iloc[i]['datetime']
                #     remark = 'EOD Sq Off'
                #     trade = {
                #         'Entry Time': entry_time,
                #         'Entry Price': entry_price,
                #         'Initial SL': initial_sl,
                #         'Exit Time': exit_time,
                #         'Exit Price': exit_price, 
                #         'Profit': entry_price - exit_price,
                #         'Remark': remark,
                #         'Trade Type': trade_type,
                #     }
                #     tradebook.append(trade)
                #     in_trade = False
                #     is_trailing_active = False

                if in_trade and trade_type == "Buy" and df.iloc[i]['l'] < initial_sl:
                    # Initial SL Hit
                    exit_price = initial_sl
                    exit_time = df.iloc[i]['datetime']
                    remark = 'Initial SL'
                    trade = {
                        'Entry Time': entry_time,
                        'Entry Price': entry_price,
                        'Initial SL': initial_sl,
                        'Exit Time': exit_time,
                        'Exit Price': exit_price, 
                        'Profit': exit_price - entry_price,
                        'Remark': remark,
                        'Trade Type': trade_type,
                    }
                    tradebook.append(trade)
                    in_trade = False
                    is_trailing_active = False

                elif in_trade and trade_type == "Sell" and df.iloc[i]['h'] > initial_sl:
                    # Initial SL Hit
                    exit_price = initial_sl
                    exit_time = df.iloc[i]['datetime']
                    remark = 'Initial SL'
                    trade = {
                        'Entry Time': entry_time,
                        'Entry Price': entry_price,
                        'Initial SL': initial_sl,
                        'Exit Time': exit_time,
                        'Exit Price': exit_price, 
                        'Profit': entry_price - exit_price,
                        'Remark': remark,
                        'Trade Type': trade_type,
                    }
                    tradebook.append(trade)
                    in_trade = False
                    is_trailing_active = False

                elif in_trade and trade_type == "Buy" and sell_signal == 1:
                    # PSAR Reversal Exit
                    exit_price = df.iloc[i]['c']
                    exit_time = df.iloc[i]['datetime']
                    remark = 'PSAR Reversal'
                    trade = {
                        'Entry Time': entry_time,
                        'Entry Price': entry_price,
                        'Initial SL': initial_sl,
                        'Exit Time': exit_time,
                        'Exit Price': exit_price, 
                        'Profit': exit_price - entry_price,
                        'Remark': remark,
                        'Trade Type': trade_type,
                    }
                    tradebook.append(trade)
                    in_trade = False
                    is_trailing_active = False

                elif in_trade and trade_type == "Sell" and buy_signal == 1:
                    # PSAR Reversal Exit
                    exit_price = df.iloc[i]['c']
                    exit_time = df.iloc[i]['datetime']
                    remark = 'PSAR Reversal'
                    trade = {
                        'Entry Time': entry_time,
                        'Entry Price': entry_price,
                        'Initial SL': initial_sl,
                        'Exit Time': exit_time,
                        'Exit Price': exit_price, 
                        'Profit': entry_price - exit_price,
                        'Remark': remark,
                        'Trade Type': trade_type,
                    }
                    tradebook.append(trade)
                    in_trade = False
                    is_trailing_active = False

                # Exit conditions
                if is_trailing_active and in_trade:
                    if trade_type == 'Buy':
                        if df.iloc[i]['c'] < df.iloc[i]['MA']:
                            #Trailing SL hit
                            exit_price = df.iloc[i]['c']
                            exit_time = df.iloc[i]['datetime']
                            remark = 'Trailing SL'
                            trade = {
                                'Entry Time': entry_time,
                                'Entry Price': entry_price,
                                'Initial SL': initial_sl,
                                'Exit Time': exit_time,
                                'Exit Price': exit_price, 
                                'Profit': exit_price - entry_price,
                                'Remark': remark,
                                'Trade Type': trade_type,
                            }
                            tradebook.append(trade)
                            in_trade = False
                            is_trailing_active = False

                    elif trade_type == 'Sell':
                        if df.iloc[i]['c'] > df.iloc[i]['MA']:
                            #Trailing SL hit
                            exit_price = df.iloc[i]['c']
                            exit_time = df.iloc[i]['datetime']
                            remark = 'Trailing SL'
                            trade = {
                                'Entry Time': entry_time,
                                'Entry Price': entry_price,
                                'Initial SL': initial_sl,
                                'Exit Time': exit_time,
                                'Exit Price': exit_price, 
                                'Profit': entry_price - exit_price,
                                'Remark': remark,
                                'Trade Type': trade_type,
                            }
                            tradebook.append(trade)
                            in_trade = False
                            is_trailing_active = False

    return pd.DataFrame(tradebook)

tb = strategy(df)

tb['Trade Year'] = tb['Entry Time'].dt.year
tb['Points'] = tb['Profit'] - 12
tb['ROI%'] = tb['Profit'] * 100 * 5 / tb['Entry Price'] # multiplication with 5 is for leverage
# tb

In [496]:
# print(tb['ROI%'].sum())

In [497]:
# df

In [498]:
# import dash
# from dash import dcc, html
# import plotly.graph_objects as go
# import pandas as pd

# # df = pd.DataFrame(bnf)

# # start_date = dt.date(2023, 1, 4)
# # end_date = dt.date(2024, 1, 4)
# # subset_df = df[(df['datetime'].dt.date >= start_date) & (df['datetime'].dt.date <= end_date)]
# # df = subset_df

# df['datetime_str'] = df['datetime'].dt.strftime('%Y/%m/%d %H:%M')

# # Create Candlestick Chart
# fig = go.Figure(data=[go.Candlestick(x=df['datetime_str'],
#                                      open=df['o'],
#                                      high=df['h'],
#                                      low=df['l'],
#                                      close=df['c'])])

# # Add PSAR as a scatter plot
# fig.add_trace(go.Scatter(x=df['datetime_str'], y=df['psar'], mode='markers', marker=dict(color='black', size=3)))

# fig.add_trace(go.Scatter(x=df['datetime_str'], y=df['MA']))

# fig.update_layout(height=1000, width=4000, title_text="PSAR")
# fig.update_xaxes(rangeslider_visible=False)

# # Set x-axis properties to display dates at regular intervals
# fig.update_xaxes(tickmode='linear', dtick=15, tickangle=45)  # Display every 1 date

# # Dash app layout
# app = dash.Dash(__name__)

# app.layout = html.Div([
#     dcc.Graph(
#         id='candlestick-chart',
#         figure=fig
#     )
# ])

# if __name__ == '__main__':
#     app.run_server(debug=True)


In [499]:
# tb

In [500]:
stats_df5 = pd.DataFrame(index=range(2017, 2024), columns=['Total ROI', 'Total Trades', 'Win Rate', 'Avg Profit% per Trade', 'Avg Loss% per Trade', 'Max Drawdown', 'ROI/DD Ratio'])
new_tb = tb
# Iterate over each year
for year in range(2017, 2024):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb['Trade Year'] == year)]
    
    # Calculate total ROI
    total_roi = year_trades['ROI%'].sum()
    
    # Calculate total number of trades
    total_trades = len(year_trades)
    
    # Calculate win rate
    win_rate = (year_trades['ROI%'] > 0).mean()*100
    
    # Calculate average profit per trade
    avg_profit = year_trades[year_trades['ROI%'] > 0]['ROI%'].mean()
    
    # Calculate average loss per trade
    avg_loss = year_trades[year_trades['ROI%'] < 0]['ROI%'].mean()
    
    # Calculate maximum drawdown
    max_drawdown = (year_trades['ROI%'].cumsum() - year_trades['ROI%'].cumsum().cummax()).min()
    
    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)
    # 
    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [total_roi, total_trades, win_rate, avg_profit, avg_loss, max_drawdown, roi_dd_ratio]

# Calculate overall statistics
overall_total_roi = stats_df5['Total ROI'].sum()
overall_total_trades = stats_df5['Total Trades'].sum()
overall_win_rate = (new_tb['ROI%'] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb['ROI%'] > 0]['ROI%'].mean()
overall_avg_loss = new_tb[new_tb['ROI%'] < 0]['ROI%'].mean()
overall_max_drawdown = (new_tb['ROI%'].cumsum() - new_tb['ROI%'].cumsum().cummax()).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc['Overall'] = [overall_total_roi, overall_total_trades, overall_win_rate, overall_avg_profit, overall_avg_loss, overall_max_drawdown, overall_roi_dd_ratio]
stats_df5


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.7542,359,28.690808,2.311559,-0.927095,-28.287567,0.026662
2018,66.307564,357,30.252101,3.226847,-1.133301,-30.676262,2.161527
2019,148.911521,356,32.303371,3.758395,-1.175535,-24.779606,6.009439
2020,336.701994,346,32.947977,7.976506,-2.478873,-63.620279,5.292369
2021,216.958581,352,33.806818,5.037421,-1.641607,-34.415581,6.30408
2022,199.643749,356,36.235955,4.224375,-1.521148,-23.167728,8.617321
2023,25.211694,366,31.147541,2.434602,-1.001321,-38.645337,0.652386
Overall,994.489304,2492.0,32.274712,4.189668,-1.392601,-63.620279,15.63164


In [492]:
# Positional 11MA, 15min TF -> Best